In [199]:
import torch
from torch.distributions import Bernoulli, Categorical, Beta, NegativeBinomial, Uniform, Normal
from tqdm import tqdm
from scipy.optimize import minimize
from scipy.optimize import differential_evolution



In [112]:
def generate_agents_var1(probability_client=0.1, buy_intention_concentration1=0.5, buy_intention_concentration2=0.5,
                    nr_visits_total_count=20, nr_visits_probs=0.8, nr_samples=1000):
    '''
    Generate agents knowing:
        - buy_intention (float in (0,1)): intention of buying
        - client (boolean): whether is a client or not
        - nr_visits (int): number of visits done beforehand
    '''
    buy_intention = Beta(buy_intention_concentration1, buy_intention_concentration2)
    client = Bernoulli(probability_client)
    nr_visits = NegativeBinomial(total_count=nr_visits_total_count, probs=nr_visits_probs)

    buy_intention_samples = buy_intention.sample((nr_samples,))
    client_samples = client.sample((nr_samples,))    
    nr_visits_samples = nr_visits.sample((nr_samples,))

    agents_tensor = torch.stack((buy_intention_samples, client_samples, nr_visits_samples), dim=1)

    return agents_tensor


In [65]:
def generate_agents_var2():
    '''
    tech = Bernoulli(0.7)
    age = Categorical(probs=torch.tensor([0.25, 0.6, 0.15]))
    discounts = Uniform(0.3, 0.8)

    tech_samples = tech.sample((10000,))
    age_samples = age.sample((10000,))
    '''
    pass

In [166]:
def will_buy_linear(agent, discount, weights, threshold=0.5):
    """
    Predict whether a client will buy something based on their characteristics.
    
    Args:
    - agent (tensor): Buy intention, whether the person is a client or not and number of visits
    - discount (float): Discount observed by the client.
    - weights (dict): Weights for each of the variables in the model.
    - threshold (float): Decision threshold for making a purchase.
    
    Returns:
    - buy_decision (bool): True if the client is predicted to buy, False otherwise.
    """
    
    weighted_sum = (
        weights['buy_intention'] * agent[0] +
        weights['client'] * agent[1] +
        weights['nr_visits'] * agent[2] / 100 +
        weights['discount'] * discount 
    )
    
    buy_probability = torch.sigmoid(torch.tensor(weighted_sum))
    
    buy_decision = buy_probability.item() > threshold
    
    return buy_decision

In [211]:
def will_buy_polynomial(agent, discount, weights, threshold=0.5):
    """
    Polynomial model for predicting purchase decision based on input features.
    
    Args:
    - agent (tensor): Buy intention, whether the person is a client or not and number of visits
    - discount (float): Discount observed by the client.
    - weights (dict): Weights for each of the variables in the model.
    - threshold (float): Decision threshold for making a purchase.
    
    Returns:
    - buy_decision (bool): True if the client is predicted to buy, False otherwise.
    """
    # de pus un polinom pt fiecare feature
    weighted_sum = (
        weights['buy_intention'] * (agent[0] ** 3) +
        weights['nr_visits'] * ((agent[2] / 100) ** 2) +
        weights['client'] * agent[1] +
        weights['discount'] * discount 
    )
    
    buy_probability = torch.sigmoid(torch.tensor(weighted_sum))
    
    buy_decision = buy_probability.item() > threshold
    
    return buy_decision


In [ ]:
def compute_buy_rate_per_day(weights, discount_sample, nr_agents_mean=17000, nr_agents_std=5000, model = 'linear', threshold=0.5):
    nr_agents_distribution = Normal(nr_agents_mean, nr_agents_std)
    # discounts_distribution = Uniform(0.3, 0.8)

    nr_agents = int(nr_agents_distribution.sample())
    agents = generate_agents_var1(nr_samples=nr_agents)
    # discount_sample = discounts_distribution.sample()

    buyers = []
    for i in range(nr_agents):
        if model == 'linear':
            buy_decision = will_buy_linear(agents[i], discount_sample, weights, threshold=threshold)
        elif model == 'polynomial':
            buy_decision = will_buy_polynomial(agents[i], discount_sample, weights, threshold=threshold)
        else:
            return "Model unrecognized"
        buyers.append(buy_decision)
        
    return sum(buyers)/len(buyers) * 100


### Check if a client is going to buy

In [ ]:
# Example weights (tune these based on your application)
weights = {
    'buy_intention': 2,
    'client': 1.5,
    'nr_visits': 0.5,
    'discount': 1,
}

# Predict whether the client will buy
ex_agent = generate_agents_var1(nr_samples=1)[0]
discounts = Uniform(0.3, 0.8)
ex_discount = discounts.sample()
buy_decision = will_buy_linear(ex_agent, ex_discount, weights)

print(f"Client will buy: {buy_decision}")


Client will buy: True


C:\Users\aledragomir\AppData\Local\Temp\ipykernel_3544\2607145893.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  buy_probability = torch.sigmoid(torch.tensor(weighted_sum))


### Check buy rate

In [237]:
weights = {
    'client': 0.05,
    'buy_intention': 0.1,
    'discount': -0.1,
    'nr_visits': 0.1
}
optimal_weights = {
    'client': 0.10336490701521983, 
    'buy_intention': -1.659139677733452, 
    'discount': 1.7634846657437528, 
    'nr_visits': -1.6603175669790198
}
optimal_weights2 = {
    'client': 0.34609360002695433, 
    'buy_intention': 2.613208217330935, 
    'discount': -0.9798212343384489, 
    'nr_visits': -2.744351905215244
}
optimal_weights3 = {
    'client': 2.114004995094768, 
    'buy_intention': 1.056666893728789, 
    'discount': -2.8883069864678266, 
    'nr_visits': -1.7351176018905805
}
# Current buy rate: 0.911925175370226

buyers = compute_buy_rate_per_day(weights=optimal_weights3, threshold=0.5, model='polynomial')
buyers

C:\Users\aledragomir\AppData\Local\Temp\ipykernel_3544\1523727861.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  buy_probability = torch.sigmoid(torch.tensor(weighted_sum))


1.0625286653416908

### Optimize weights to reach 0.8% buy rate

In [232]:
def optimize_weights(nr_agents, target_sum, threshold=0.5, model = 'linear'):
    """
    Adjust the weights to achieve the target sum of buy_decisions across the samples.
    
    Args:
    - num_samples (int): Number of samples to generate.
    - target_sum (int): Target sum of buy_decisions.
    - initial_weights (dict): Initial weights for the model.
    - threshold (float): Decision threshold for making a purchase.
    
    Returns:
    - optimal_weights (dict): Weights adjusted to achieve the target sum.
    """
    
    def objective_function(weights_array):
        weights = {
            'client': weights_array[0],
            'buy_intention': weights_array[1],
            'discount': weights_array[2],
            'nr_visits': weights_array[3]
        }
        
        # Generate samples and calculate the sum of buy_decisions
        agents = generate_agents_var1(nr_samples=nr_agents)
        discounts_distribution = Uniform(0.3, 0.8)
        

        buy_decisions = []
        for i in range(nr_agents):
            discount_sample = discounts_distribution.sample()
            if model == 'linear':
                buy_decision = will_buy_linear(agents[i], discount_sample, weights, threshold=threshold)    
            elif model == 'polynomial':
                buy_decision = will_buy_polynomial(agents[i], discount_sample, weights, threshold=threshold)
            buy_decisions.append(buy_decision)
        
        sum_buy_decisions = sum(buy_decisions)
        
        loss = (sum_buy_decisions - target_sum) / len(buy_decisions) 
        print(f"Current loss: {loss}, Weights: {weights}")  
        buy_rate = compute_buy_rate_per_day(weights=weights, model=model)
        print(f"Current buy rate: {buy_rate}")
        return loss
    
    # # Initial weights in array form
    # initial_weights_array = [
    #     initial_weights['client'],
    #     initial_weights['buy_intention'],
    #     initial_weights['discount'],
    #     initial_weights['nr_visits']
    # ]
    
    # Optimize the weights to minimize the objective function
    # result = minimize(objective_function, initial_weights_array, method='Powell')
    # Define bounds for each weight
    bounds = [(-3, 3), (-3, 3), (-3, 3), (-3, 3)]

    # Run the differential evolution algorithm
    result = differential_evolution(objective_function, bounds)
    
    # Convert the result back to a dictionary
    optimal_weights = {
        'client': result.x[0],
        'buy_intention': result.x[1],
        'discount': result.x[2],
        'nr_visits': result.x[3]
    }
    
    return optimal_weights

# # Initial weights (can be random or based on intuition)
# initial_weights = {
#     'client': 1,
#     'buy_intention': 0.58,
#     'discount': 0.35,
#     'nr_visits': 0.5
# }



In [233]:
optimal_weights = optimize_weights(nr_agents=10000, target_sum=80, threshold=0.5, model='polynomial')

print("Optimal Weights:", optimal_weights)

C:\Users\aledragomir\AppData\Local\Temp\ipykernel_3544\1523727861.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  buy_probability = torch.sigmoid(torch.tensor(weighted_sum))


Current loss: 0.2327, Weights: {'client': 1.3185936343733788, 'buy_intention': -2.505018444327088, 'discount': -2.9104386569795224, 'nr_visits': 2.04223998611842}
Current buy rate: 15.841584158415841
Current loss: 0.992, Weights: {'client': 2.660489894719282, 'buy_intention': 2.5930214099375584, 'discount': 0.5088327413846947, 'nr_visits': 0.021609047993487396}
Current buy rate: 100.0
Current loss: -0.008, Weights: {'client': -2.450152298846887, 'buy_intention': 0.3258241949781717, 'discount': -1.8729081963876695, 'nr_visits': -2.9817531311631407}
Current buy rate: 0.0
Current loss: 0.0464, Weights: {'client': -2.8349795152125274, 'buy_intention': 1.4310098624329213, 'discount': -0.7160874915240494, 'nr_visits': -2.036574121505409}
Current buy rate: 5.753870078461865
Current loss: 0.4054, Weights: {'client': -0.8100801162813325, 'buy_intention': -2.741062477934441, 'discount': 0.2451953656421615, 'nr_visits': 0.14450006881682298}
Current buy rate: 41.63994743758213
Current loss: 0.9868

KeyboardInterrupt: 

## Multi armed bandits

10 zile, cate un esantion de agenti in fiecare zi
3 discounturi (bandits)